# Advanced NumPy

In [1]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
np.set_printoptions(precision=4)
import sys

## ndarray object internals

### NumPy dtype hierarchy

In [2]:
ints = np.ones(10, dtype=np.uint16)
floats = np.ones(10, dtype=np.float32)
np.issubdtype(ints.dtype, np.integer)
np.issubdtype(floats.dtype, np.floating)

True

In [3]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

## Advanced array manipulation

### Reshaping arrays

In [ ]:
arr = np.arange(8)
arr
arr.reshape((4, 2))

In [ ]:
arr.reshape((4, 2)).reshape((2, 4))

In [ ]:
arr = np.arange(15)
arr.reshape((5, -1))

In [ ]:
other_arr = np.ones((3, 5))
other_arr.shape
arr.reshape(other_arr.shape)

In [ ]:
arr = np.arange(15).reshape((5, 3))
arr
arr.ravel()

In [ ]:
arr.flatten()

### C vs. Fortran order

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr
arr.ravel()
arr.ravel('F')

### Concatenating and splitting arrays

In [ ]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])
arr2 = np.array([[7, 8, 9], [10, 11, 12]])
np.concatenate([arr1, arr2], axis=0)
np.concatenate([arr1, arr2], axis=1)

In [ ]:
np.vstack((arr1, arr2))
np.hstack((arr1, arr2))

In [ ]:
from numpy.random import randn
arr = randn(5, 2)
arr
first, second, third = np.split(arr, [1, 3])
first
second
third

#### Stacking helpers: 

In [ ]:
arr = np.arange(6)
arr1 = arr.reshape((3, 2))
arr2 = randn(3, 2)
np.r_[arr1, arr2]
np.c_[np.r_[arr1, arr2], arr]

In [ ]:
np.c_[1:6, -10:-5]

### Repeating elements: tile and repeat

In [4]:
arr = np.arange(3)
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

In [5]:
arr.repeat([2, 3, 4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

In [15]:
arr = randn(2, 2)

In [18]:
arr
arr.repeat(2, axis=0)

array([[-0.3546,  0.4205],
       [-0.3546,  0.4205],
       [ 0.1212,  0.0334],
       [ 0.1212,  0.0334]])

In [7]:
arr.repeat([2, 3], axis=0)
arr.repeat([2, 3], axis=1)

array([[ 0.363 ,  0.363 ,  0.9671,  0.9671,  0.9671],
       [-0.9217, -0.9217,  0.8709,  0.8709,  0.8709]])

In [8]:
arr
np.tile(arr, 2)

array([[ 0.363 ,  0.9671,  0.363 ,  0.9671],
       [-0.9217,  0.8709, -0.9217,  0.8709]])

In [9]:
arr
np.tile(arr, (2, 1))
np.tile(arr, (3, 2))

array([[ 0.363 ,  0.9671,  0.363 ,  0.9671],
       [-0.9217,  0.8709, -0.9217,  0.8709],
       [ 0.363 ,  0.9671,  0.363 ,  0.9671],
       [-0.9217,  0.8709, -0.9217,  0.8709],
       [ 0.363 ,  0.9671,  0.363 ,  0.9671],
       [-0.9217,  0.8709, -0.9217,  0.8709]])

### Fancy indexing equivalents: take and put

In [ ]:
arr = np.arange(10) * 100
inds = [7, 1, 2, 6]
arr[inds]

In [ ]:
arr.take(inds)
arr.put(inds, 42)
arr
arr.put(inds, [40, 41, 42, 43])
arr

In [ ]:
inds = [2, 0, 2, 1]
arr = randn(2, 4)
arr
arr.take(inds, axis=1)

## Broadcasting

In [22]:
arr = np.arange(5)
arr
arr * 4

array([ 0,  4,  8, 12, 16])

In [31]:
arr = randn(4, 3)
print(arr)
print(arr.mean(0))
demeaned = arr - arr.mean(0)
demeaned
demeaned.mean(0)

[[-2.4046  0.3436 -0.8001]
 [ 0.3556 -0.7967 -0.5822]
 [-0.1554 -1.7439  0.4186]
 [-2.0469  1.8307  1.0727]]
[-1.0628 -0.0916  0.0273]


array([  0.0000e+00,   5.5511e-17,   0.0000e+00])

In [ ]:
arr
row_means = arr.mean(1)
row_means.reshape((4, 1))
demeaned = arr - row_means.reshape((4, 1))
demeaned.mean(1)

### Broadcasting over other axes

In [ ]:
arr - arr.mean(1)

In [ ]:
arr - arr.mean(1).reshape((4, 1))

In [ ]:
arr = np.zeros((4, 4))
arr_3d = arr[:, np.newaxis, :]
arr_3d.shape

In [42]:
arr_1d = np.random.normal(size=3)
arr_1d[np.newaxis, :]

(3,)
(3, 1)


array([[-1.1304, -0.9739, -1.8916]])

In [ ]:
arr = randn(3, 4, 5)
depth_means = arr.mean(2)
depth_means
demeaned = arr - depth_means[:, :, np.newaxis]
demeaned.mean(2)

In [ ]:
def demean_axis(arr, axis=0):
    means = arr.mean(axis)

    # This generalized things like [:, :, np.newaxis] to N dimensions
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]

### Setting array values by broadcasting

In [ ]:
arr = np.zeros((4, 3))
arr[:] = 5
arr

In [44]:
col = np.array([1.28, -0.42, 0.44, 1.6])
arr[:] = col[:, np.newaxis]
arr[:2] = [[-1.37], [0.509]]
arr

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])

## Advanced ufunc usage

### Ufunc instance methods

In [ ]:
arr = np.arange(10)
np.add.reduce(arr)
arr.sum()

In [ ]:
np.random.seed(12346)

In [45]:
arr = randn(5, 5)

In [49]:
arr[::2].sort(1) # sort a few rows
print(arr)
print(arr[:, :-1])
print(arr[:, :-1] < arr[:, 1:])
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis=1)

[[-1.9403 -1.7164 -0.7516  0.5822  2.0498]
 [-0.7149  0.3988 -0.1267  0.6504  0.8249]
 [-0.9796 -0.9735 -0.4562 -0.4513  0.5595]
 [-1.4219 -1.1024 -0.8647  0.6786 -0.1095]
 [-0.713  -0.1247 -0.0721  0.5786  0.9591]]
[[-1.9403 -1.7164 -0.7516  0.5822]
 [-0.7149  0.3988 -0.1267  0.6504]
 [-0.9796 -0.9735 -0.4562 -0.4513]
 [-1.4219 -1.1024 -0.8647  0.6786]
 [-0.713  -0.1247 -0.0721  0.5786]]
[[ True  True  True  True]
 [ True False  True  True]
 [ True  True  True  True]
 [ True  True  True False]
 [ True  True  True  True]]


array([ True, False,  True, False,  True], dtype=bool)

In [ ]:
arr = np.arange(15).reshape((3, 5))
np.add.accumulate(arr, axis=1)

In [50]:
arr = np.arange(3).repeat([1, 2, 2])
arr
np.multiply.outer(arr, np.arange(5))

array([[0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

In [51]:
result = np.subtract.outer(randn(3, 4), randn(5))
result.shape

(3, 4, 5)

In [ ]:
arr = np.arange(10)
np.add.reduceat(arr, [0, 5, 8])

In [ ]:
arr = np.multiply.outer(np.arange(4), np.arange(5))
arr
np.add.reduceat(arr, [0, 2, 4], axis=1)

### Custom ufuncs

In [ ]:
def add_elements(x, y):
    return x + y
add_them = np.frompyfunc(add_elements, 2, 1)
add_them(np.arange(8), np.arange(8))

In [ ]:
add_them = np.vectorize(add_elements, otypes=[np.float64])
add_them(np.arange(8), np.arange(8))

In [ ]:
arr = randn(10000)
%timeit add_them(arr, arr)
%timeit np.add(arr, arr)

## Structured and record arrays

In [ ]:
dtype = [('x', np.float64), ('y', np.int32)]
sarr = np.array([(1.5, 6), (np.pi, -2)], dtype=dtype)
sarr

In [ ]:
sarr[0]
sarr[0]['y']

In [ ]:
sarr['x']

### Nested dtypes and multidimensional fields

In [ ]:
dtype = [('x', np.int64, 3), ('y', np.int32)]
arr = np.zeros(4, dtype=dtype)
arr

In [ ]:
arr[0]['x']

In [ ]:
arr['x']

In [ ]:
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), ('y', np.int32)]
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype=dtype)
data['x']
data['y']
data['x']['a']

### Why use structured arrays?

### Structured array manipulations: numpy.lib.recfunctions

## More about sorting

In [ ]:
arr = randn(6)
arr.sort()
arr

In [ ]:
arr = randn(3, 5)
arr
arr[:, 0].sort()  # Sort first column values in-place
arr

In [ ]:
arr = randn(5)
arr
np.sort(arr)
arr

In [4]:
arr = randn(3, 5)
arr
arr.sort(axis=1)
arr

TypeError: getattr expected at least 2 arguments, got 1

In [ ]:
arr[:, ::-1]

### Indirect sorts: argsort and lexsort

In [ ]:
values = np.array([5, 0, 1, 3, 2])
indexer = values.argsort()
indexer
values[indexer]

In [ ]:
arr = randn(3, 5)
arr[0] = values
arr
arr[:, arr[0].argsort()]

In [ ]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])
sorter = np.lexsort((first_name, last_name))
zip(last_name[sorter], first_name[sorter])

### Alternate sort algorithms

In [ ]:
values = np.array(['2:first', '2:second', '1:first', '1:second', '1:third'])
key = np.array([2, 2, 1, 1, 1])
indexer = key.argsort(kind='mergesort')
indexer
values.take(indexer)

### numpy.searchsorted: Finding elements in a sorted array

In [ ]:
arr = np.array([0, 1, 7, 12, 15])
arr.searchsorted(9)

In [ ]:
arr.searchsorted([0, 8, 11, 16])

In [ ]:
arr = np.array([0, 0, 0, 1, 1, 1, 1])
arr.searchsorted([0, 1])
arr.searchsorted([0, 1], side='right')

In [ ]:
data = np.floor(np.random.uniform(0, 10000, size=50))
bins = np.array([0, 100, 1000, 5000, 10000])
data

In [ ]:
labels = bins.searchsorted(data)
labels

In [ ]:
Series(data).groupby(labels).mean()

In [ ]:
np.digitize(data, bins)

## NumPy matrix class

In [ ]:
X =  np.array([[ 8.82768214,  3.82222409, -1.14276475,  2.04411587],
               [ 3.82222409,  6.75272284,  0.83909108,  2.08293758],
               [-1.14276475,  0.83909108,  5.01690521,  0.79573241],
               [ 2.04411587,  2.08293758,  0.79573241,  6.24095859]])
X[:, 0]  # one-dimensional
y = X[:, :1]  # two-dimensional by slicing
X
y

In [ ]:
np.dot(y.T, np.dot(X, y))

In [ ]:
Xm = np.matrix(X)
ym = Xm[:, 0]
Xm
ym
ym.T * Xm * ym

In [ ]:
Xm.I * X

## Advanced array input and output

### Memory-mapped files

In [ ]:
mmap = np.memmap('mymmap', dtype='float64', mode='w+', shape=(10000, 10000))
mmap

In [ ]:
section = mmap[:5]

In [ ]:
section[:] = np.random.randn(5, 10000)
mmap.flush()
mmap
del mmap

In [ ]:
mmap = np.memmap('mymmap', dtype='float64', shape=(10000, 10000))
mmap

In [ ]:
%xdel mmap
!rm mymmap

### HDF5 and other array storage options

## Performance tips

### The importance of contiguous memory

In [ ]:
arr_c = np.ones((1000, 1000), order='C')
arr_f = np.ones((1000, 1000), order='F')
arr_c.flags
arr_f.flags
arr_f.flags.f_contiguous

In [ ]:
%timeit arr_c.sum(1)
%timeit arr_f.sum(1)

In [ ]:
arr_f.copy('C').flags

In [ ]:
arr_c[:50].flags.contiguous
arr_c[:, :50].flags

In [ ]:
%xdel arr_c
%xdel arr_f
%cd ..

## Other speed options: Cython, f2py, C

```cython
from numpy cimport ndarray, float64_t

def sum_elements(ndarray[float64_t] arr):
    cdef Py_ssize_t i, n = len(arr)
    cdef float64_t result = 0

    for i in range(n):
        result += arr[i]

    return result
```